# Projeto IC

Download e import das bibliotecas necessárias para a execução das rotinas em python

In [1]:
import time
import numpy as np
import re
import h5py
import meshio
import sys

Lendo a malha utilizando meshio, muito mais rápido que o MSHFileParse

In [2]:
def Read_Mesh(filePath, dimension):
    inicio = time.time()
    mesh = meshio.read(filePath)
    fim = time.time()
    print(f"Tempo de leitura da malha: {fim - inicio:.4f} segundos")

    nodes = np.round(mesh.points, 6)
    elem = 2**dimension
    # Inicializa um array para conectividades
    elements = np.empty((0, elem), dtype=int)

    # Itera sobre os tipos de células no arquivo
    for cell in mesh.cells:
        if cell.data.shape[1] == elem: 
            elements = np.concatenate((elements, cell.data)) + 1
    
    return nodes, elements

Criando a classe que armazena a malha e faz a divisão para a malha grossa e malha dual

OBS: agilizar o divide_mesh

In [3]:
class MeshDataStructure:
  def __init__(self, nodes_file: str, connectivities_file: str, dimension = None):
    self.dimension = dimension
    self.nodes_file = nodes_file
    self.connectivities_file = connectivities_file
    self.nodes = None
    self.connectivities = None
    self.centroids = None
    self.coords_centroids = None
    self.nHorizontal = None
    self.nVertical = None
    self.nAltura = None
    self.coord_grossa = None
    self.grossa_centroids_array = None
    self.vertices = None
    self.arestas = None
    self.faces = None
    self.internos = None
    self.nos_na_proxima_hierarquia = None
    self.conexoes_na_proxima_hierarquia = None
    self.nX = None
    self.nY = None
    self.nZ = None
    self.Grossa_Index = None

  def set_nodes(self, nodes=None):
    if nodes is None:
      nos = np.array([[1, 2, 3]])
      with h5py.File(self.nodes_file, 'r') as file:
          grupos = []
          for group in file.keys():
              if("entity" in group):
                grupos.append(group)

          for grupo_nome in grupos:
            grupo = file[grupo_nome]
            for dataset in grupo.keys():
                dados = []
                try:
                  dados = grupo[dataset][:]
                  dados = np.round(dados, 6)
                  nos = np.concatenate((nos, dados))
                except:
                  dados = grupo[dataset]
          nos = nos[1:]
          self.nodes = nos
    else:
      self.nodes = nodes

  def set_connectivities(self, connections = None):
    if connections is None:
      conexao = np.array([[1,2,3,4]])
      with h5py.File(self.connectivities_file, 'r') as file:
          grupos = []
          for group in file.keys():
              if("entity" in group):
                grupos.append(group)

          for grupo_nome in grupos:
            grupo = file[grupo_nome]
            for dataset in grupo.keys():
                dados = []
                try:
                  dados = grupo[dataset][:]
                  if(isinstance(dados[0], np.ndarray)):
                    if(len(dados[0]) == 4):
                      conexao = np.concatenate((conexao, dados))
                except:
                  dados = grupo[dataset]
          conexao = conexao[1:]
          self.connectivities = conexao
    else:
      self.connectivities = connections
  
  def get_centroids(self):
    connectivities = np.array(self.connectivities) - 1
    nodes = np.array(self.nodes)

    coords = nodes[connectivities]
    sum_coords = np.sum(coords, axis=1)
    centroids = sum_coords / 4.0

    centroids_rounded = np.round(centroids, 6)
    indices = np.arange(len(centroids)).reshape(-1, 1)
    centroids_with_indices = np.hstack((centroids_rounded, indices))

    self.centroids = centroids_with_indices
    self.coords_centroids = coords

  def sort_centroids(self, centroids):
    centroids = np.asarray(centroids)
    indices_ordenados = np.lexsort((centroids[:, 2], centroids[:, 1], centroids[:, 0]))
    return centroids[indices_ordenados]


  def get_dimensions(self):
    error = 0.0005
    firstX = self.centroids[0, 0]
    firstY = self.centroids[0, 1]
    if self.dimension == 3:
      firstZ = self.centroids[0, 2]

      nVertical = np.sum((self.centroids[:, 0] >= firstX - error) & (self.centroids[:, 0] <= firstX + error) & (self.centroids[:, 2] >= firstZ - error) & (self.centroids[:, 2] <= firstZ + error))
      nHorizontal = np.sum((self.centroids[:, 1] >= firstY - error) & (self.centroids[:, 1] <= firstY + error) & (self.centroids[:, 2] >= firstZ - error) & (self.centroids[:, 2] <= firstZ + error))
      nAltura = np.sum((self.centroids[:, 1] >= firstY - error) & (self.centroids[:, 1] <= firstY + error) & (self.centroids[:, 0] >= firstX - error) & (self.centroids[:, 0] <= firstX + error))
      self.nAltura = nAltura

    else:
      nVertical = np.sum((self.centroids[:, 1] >= firstX - error) & (self.centroids[:, 1] <= firstX + error))
      nHorizontal = np.sum((self.centroids[:, 1] >= firstY - error) & (self.centroids[:, 1] <= firstY + error))

    self.nHorizontal = nHorizontal
    self.nVertical = nVertical

  # NX E NY DEVEM SER ÍMPARES!
  def divide_mesh(self, centroids, H, V, nX, nY, A = None, nZ = None):
    
    if (nX % 2 == 0) or (nY % 2 == 0):
      print("Os valores nX ou nY são pares")
      return

    if self.dimension == 3:
      if A is None or nZ is None:
        print("Insira um valor de Altura (A) ou da redução nZ")
        return
      if nZ % 2 == 0:
        print("O valor A não é múltiplo de nZ, ou nZ não é ímpar")
        return

    coord_grossa = np.empty(centroids.shape[0], dtype=object)
    ja_foi_mapeado = set()

    # calcula quantos nós serão adicionados em cada lado caso H % nX != 0, e o mesmo para as outras dimensões
    rest_x = (H % nX)
    add_x_left = rest_x // 2
    add_x_right = rest_x - add_x_left
    rest_y = (V % nY)
    add_y_left = rest_y // 2
    add_y_right = rest_y - add_y_left

    if self.dimension == 3:
      rest_z = (A % nZ)
      add_z_left = rest_z // 2
      add_z_right = rest_z - add_z_left

      grossa_centroids_array = np.empty((H // nX) * (V // nY) * (A // nZ), dtype=object)

      grossa_centroids_array[:] = [np.array([], dtype=int) for _ in range((H // nX) * (V // nY) * (A // nZ))]

      vertices = np.zeros((H // nX) * (V // nY) * (A // nZ), dtype=int)
      arestas = np.empty((H // nX) * (V // nY) * (A // nZ), dtype=object)
      faces = np.empty((H // nX) * (V // nY) * (A // nZ), dtype=object)
      internos = np.empty((H // nX) * (V // nY) * (A // nZ), dtype=object)

      arestas[:] = [np.array([], dtype=int) for _ in range((H // nX) * (V // nY) * (A // nZ))]
      faces[:] = [np.array([], dtype=int) for _ in range((H // nX) * (V // nY) * (A // nZ))]
      internos[:] = [np.array([], dtype=int) for _ in range((H // nX) * (V // nY) * (A // nZ))]

      # Cria uma grade de valores de i, j e k
      i_vals = np.arange(H)
      j_vals = np.arange(V)
      k_vals = np.arange(A)
      i_grid, j_grid, k_grid = np.meshgrid(i_vals, j_vals, k_vals, indexing='ij')

      # Calcula indX, indY e indZ vetorizados
      indX = i_grid // nX
      indY = j_grid // nY
      indZ = k_grid // nZ
      
      # Se o tamanho não for divisível pelo fator de redução, temos que alterar a distribuição dos nós 
      # (Só são alterados se tiver que adicionar na esquerda ou direita)
      # Alterando indX
      if rest_x != 0:
        for i in range(add_x_left):
          indX[nX + i] = 0
        for i in range(nX + add_x_left, H - add_x_right, nX):
          for j in range(0, add_x_left):
            indX[i + nX - (j + 1)] = indX[i]
        for i in range(add_x_right):
          indX[-(i+1)] = indX[-nX]

      # Alterando indY
      if rest_y != 0:
        for x in range(H):
          for i in range(add_y_left):
            indY[x][nY + i] = 0
          for i in range(nY + add_y_left, V - add_y_right, nY):
            for j in range(add_y_left):
              indY[x][i + nY - (j + 1)] = indY[x][i]
          for i in range(add_y_right):
            indY[x][-(i+1)] = indY[x][-nY]

      #Alterando indZ
      if rest_z != 0:
        for x in range(H):
          for v in range(V):
            for i in range(add_z_left):
              indZ[x][v][nZ + i] = 0
            for i in range(nZ + add_z_left, A - add_z_right, nZ):
              for j in range(0, add_z_left):
                indZ[x][v][i + nZ - (j + 1)] = indZ[x][v][i]
            for i in range(add_z_right):
              indZ[x][v][-(i+1)] = indZ[x][v][-nZ]

      # Calcula os nós
      nodes = k_grid + j_grid * A + i_grid * A * V
      
      # Atribui a coordenada grossa para cada nó
      coord_grossa[nodes.flatten()] = list(zip(indX.flatten(), indY.flatten(), indZ.flatten()))

      # Prepara grossa_centroids_array usando uma lista de compreensão
      grossa_centroids_array = np.empty((H // nX) * (V // nY) * (A // nZ), dtype=object)
      grossa_centroids_array[:] = [np.array([], dtype=int) for _ in range((H // nX) * (V // nY) * (A // nZ))]

      # Atualiza grossa_centroids_array com os nós, de forma vetorizada
      self.Grossa_Index = np.arange(H * V * A)
      for indX_val, indY_val, indZ_val, node in zip(indX.flatten(), indY.flatten(), indZ.flatten(), nodes.flatten()):  
          idx = indZ_val + indY_val * (A // nZ) + indX_val * (A // nZ) * (V // nY)
          self.Grossa_Index[node] = idx
          grossa_centroids_array[idx] = np.append(grossa_centroids_array[idx], node)
        
      # Calcula x, y e z para cada índice
      indices = np.arange((H // nX) * (V // nY) * (A // nZ))
      y = indices // (A // nZ) % (V // nY)
      z = indices % (A // nZ)
      x = indices // ((A // nZ) * (V // nY))

      # Calcula os vértices para cada condição
      # Pontas
      vertices[(x == 0) & (y == 0) & (z == 0)] = [grossa_centroids_array[i][0] for i in indices[(x == 0) & (y == 0) & (z == 0)]]
      vertices[(x == (H // nX - 1)) & (y == 0) & (z == 0)] = [grossa_centroids_array[i][((nZ + add_z_left) * (nY + add_y_left) * (nX + add_x_right - 1))] for i in indices[(x == (H // nX - 1)) & (y == 0) & (z == 0)]]
      vertices[(x == 0) & (y == (V // nY - 1)) & (z == 0)] = [grossa_centroids_array[i][((nZ + add_z_left) * (nY + add_y_right - 1))] for i in indices[(x == 0) & (y == (V // nY - 1)) & (z == 0)]]
      vertices[(x == (H // nX - 1)) & (y == (V // nY - 1)) & (z == 0)] = [grossa_centroids_array[i][((nZ + add_z_left) * (nY + add_y_right) * (nX + add_x_right - 1)) + ((nZ + add_z_left) * (nY + add_y_right - 1))] for i in indices[(x == (H // nX - 1)) & (y == (V // nY - 1)) & (z == 0)]]
      vertices[(x == 0) & (y == 0) & (z == (A // nZ - 1))] = [grossa_centroids_array[i][(nZ + add_z_right) - 1] for i in indices[(x == 0) & (y == 0) & (z == (A // nZ - 1))]]
      vertices[(x == (H // nX - 1)) & (y == 0) & (z == (A // nZ - 1))] = [grossa_centroids_array[i][((nY + add_y_left) * (nZ + add_z_right) * (nX + add_x_right - 1)) + (nZ + add_z_right - 1)] for i in indices[(x == (H // nX - 1)) & (y == 0) & (z == (A // nZ - 1))]]
      vertices[(x == 0) & (y == (V // nY - 1)) & (z == (A // nZ - 1))] = [grossa_centroids_array[i][(nZ + add_z_right) * (nY + add_y_right) - 1] for i in indices[(x == 0) & (y == (V // nY - 1)) & (z == (A // nZ - 1))]]
      vertices[(x == (H // nX - 1)) & (y == (V // nY - 1)) & (z == (A // nZ - 1))] = [grossa_centroids_array[i][-1] for i in indices[(x == (H // nX - 1)) & (y == (V // nY - 1)) & (z == (A // nZ - 1))]]
      # Intermediários bordas
      vertices[(x == 0) & (y > 0) & (y < (V // nY - 1)) & (z == 0)] = [grossa_centroids_array[i][(nZ + add_z_left) * (nY // 2)] for i in indices[(x == 0) & (y > 0) & (y < (V // nY - 1)) & (z == 0)]]
      vertices[(x > 0) & (x < (H // nX - 1)) & (y == 0) & (z == 0)] = [grossa_centroids_array[i][(nZ + add_z_left) * (nY + add_y_left) * (nX // 2)] for i in indices[(x > 0) & (x < (H // nX - 1)) & (y == 0) & (z == 0)]]
      vertices[(x > 0) & (x < (H // nX - 1)) & (y == (V // nY - 1)) & (z == 0)] = [grossa_centroids_array[i][(nZ + add_z_left) * (nY + add_y_right) * (nX // 2) + ((nZ + add_z_left) * (nY + add_y_right - 1))] for i in indices[(x > 0) & (x < (H // nX - 1)) & (y == (V // nY - 1)) & (z == 0)]]
      vertices[(x == (H // nX - 1)) & (y > 0) & (y < (V // nY - 1)) & (z == 0)] = [grossa_centroids_array[i][((nZ + add_z_left) * nY * (nX + add_x_right - 1) + ((nZ + add_z_left) * (nY // 2)))] for i in indices[(x == (H // nX - 1)) & (y > 0) & (y < (V // nY - 1)) & (z == 0)]]
      vertices[(x == 0) & (y > 0) & (y < (V // nY - 1)) & (z == (A // nZ - 1))] = [grossa_centroids_array[i][((nZ + add_z_right) * (nY // 2)) + ((nZ + add_z_right) - 1)] for i in indices[(x == 0) & (y > 0) & (y < (V // nY - 1)) & (z == (A // nZ - 1))]]
      vertices[(x > 0) & (x < (H // nX - 1)) & (y == 0) & (z == (A // nZ - 1))] = [grossa_centroids_array[i][((nZ + add_z_right) * (nY + add_y_left) * (nX // 2)) + (nZ + add_z_right - 1)] for i in indices[(x > 0) & (x < (H // nX - 1)) & (y == 0) & (z == (A // nZ - 1))]]
      vertices[(x > 0) & (x < (H // nX - 1)) & (y == (V // nY - 1)) & (z == (A // nZ - 1))] = [grossa_centroids_array[i][((nZ + add_z_right) * (nY + add_y_right) * (nX // 2)) + ((nZ + add_z_right) * (nY + add_y_right) - 1)] for i in indices[(x > 0) & (x < (H // nX - 1)) & (y == (V // nY - 1)) & (z == (A // nZ - 1))]]
      vertices[(x == (H // nX - 1)) & (y > 0) & (y < (V // nY - 1)) & (z == (A // nZ - 1))] = [grossa_centroids_array[i][((nZ + add_z_right) * nY * (nX + add_x_right - 1)) + ((nZ + add_z_right) * (nY // 2)) + (nZ + add_z_right - 1)] for i in indices[(x == (H // nX - 1)) & (y > 0) & (y < (V // nY - 1)) & (z == (A // nZ - 1))]]
      vertices[(x == 0) & (y == 0) & (z < (A // nZ - 1)) & (z > 0)] = [grossa_centroids_array[i][nZ // 2] for i in indices[(x == 0) & (y == 0) & (z < (A // nZ - 1)) & (z > 0)]]
      vertices[(x == 0) & (y == (V // nY - 1)) & (z < (A // nZ - 1)) & (z > 0)] = [grossa_centroids_array[i][(nZ * (nY + add_y_right)) - (nZ // 2 + 1)] for i in indices[(x == 0) & (y == (V // nY - 1)) & (z < (A // nZ - 1)) & (z > 0)]]
      vertices[(x == (H // nX - 1)) & (y == 0) & (z < (A // nZ - 1)) & (z > 0)] = [grossa_centroids_array[i][(nZ * (nY + add_y_left) * (nX + add_x_right - 1)) + (nZ // 2)] for i in indices[(x == (H // nX - 1)) & (y == 0) & (z < (A // nZ - 1)) & (z > 0)]]
      vertices[(x == (H // nX - 1)) & (y == (V // nY - 1)) & (z < (A // nZ - 1)) & (z > 0)] = [grossa_centroids_array[i][(nZ * (nY + add_y_right) * (nX + add_x_right)) - (nZ // 2 + 1)] for i in indices[(x == (H // nX - 1)) & (y == (V // nY - 1)) & (z < (A // nZ - 1)) & (z > 0)]]
      # Intermediários laterais (Faces)
      vertices[(x == 0) & (y > 0) & (y < (V // nY - 1)) & (z > 0) & (z < (A // nZ - 1))] = [grossa_centroids_array[i][(nZ * nY) // 2] for i in indices[(x == 0) & (y > 0) & (y < (V // nY - 1)) & (z > 0) & (z < (A // nZ - 1))]]
      vertices[(x == (H // nX - 1)) & (y > 0) & (y < (V // nY - 1)) & (z > 0) & (z < (A // nZ - 1))] = [grossa_centroids_array[i][(nZ * nY * (nX + add_x_right - 1)) + (nZ * (nY // 2) + (nZ // 2))] for i in indices[(x == (H // nX - 1)) & (y > 0) & (y < (V // nY - 1)) & (z > 0) & (z < (A // nZ - 1))]]
      vertices[(x > 0) & (x < (H // nX - 1)) & (y == 0) & (z > 0) & (z < (A // nZ - 1))] = [grossa_centroids_array[i][(nZ * (nY + add_y_left) * (nX // 2)) + (nZ // 2)] for i in indices[(x > 0) & (x < (H // nX - 1)) & (y == 0) & (z > 0) & (z < (A // nZ - 1))]]
      vertices[(x > 0) & (x < (H // nX - 1)) & (y == (V // nY - 1)) & (z > 0) & (z < (A // nZ - 1))] = [grossa_centroids_array[i][(nZ * (nY + add_y_right) * (nX // 2 + 1)) - (nZ // 2 + 1)] for i in indices[(x > 0) & (x < (H // nX - 1)) & (y == (V // nY - 1)) & (z > 0) & (z < (A // nZ - 1))]]
      vertices[(x > 0) & (x < (H // nX - 1)) & (y < (V // nY - 1)) & (y > 0) & (z == 0)] = [grossa_centroids_array[i][((nZ + add_z_left) * nY * (nX // 2)) + ((nZ + add_z_left) * (nY // 2))] for i in indices[(x > 0) & (x < (H // nX - 1)) & (y < (V // nY - 1)) & (y > 0) & (z == 0)]]
      vertices[(x > 0) & (x < (H // nX - 1)) & (y < (V // nY - 1)) & (y > 0) & (z == (A // nZ - 1))] = [grossa_centroids_array[i][((nZ + add_z_right) * nY * (nX // 2)) + ((nZ + add_z_right) * (nY // 2)) + nZ + add_z_right - 1] for i in indices[(x > 0) & (x < (H // nX - 1)) & (y < (V // nY - 1)) & (y > 0) & (z == (A // nZ - 1))]]
      # Intermediários internos
      vertices[(x > 0) & (x < (H // nX - 1)) & (y > 0) & (y < (V // nY - 1)) & (z > 0) & (z < (A // nZ - 1))] = [grossa_centroids_array[i][(nZ * nY * nX) // 2] for i in indices[(x > 0) & (x < (H // nX - 1)) & (y > 0) & (y < (V // nY - 1)) & (z > 0) & (z < (A // nZ - 1))]]

      # Adiciona valores ao conjunto
      ja_foi_mapeado = set(vertices)
      vertices_borda = []

      # Preciso pegar as arestas
      # Borda x = 0 y = 0
      for index in range(0, A):
        if index in ja_foi_mapeado:
          vertices_borda.append(index)
          continue
        ja_foi_mapeado.add(index)
        pertence_na_grossa = coord_grossa[index]
        index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))

      # Borda x = 0 z = 0
      for index in range(0, V*A, A):
        if index in ja_foi_mapeado:
          vertices_borda.append(index)
          continue
        ja_foi_mapeado.add(index)
        pertence_na_grossa = coord_grossa[index]
        index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))

      # Borda x = 0 y = V
      for index in range((V - 1)*A, V * A):
        if index in ja_foi_mapeado:
          vertices_borda.append(index)
          continue
        ja_foi_mapeado.add(index)
        pertence_na_grossa = coord_grossa[index]
        index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))

      # Borda x = 0 z = A
      for index in range(A - 1, V * A, A):
        if index in ja_foi_mapeado:
          vertices_borda.append(index)
          continue
        ja_foi_mapeado.add(index)
        pertence_na_grossa = coord_grossa[index]
        index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))

      # Borda x = H y = 0
      for index in range(V * A * (H - 1), (V * A * (H - 1)) + A):
        if index in ja_foi_mapeado:
          vertices_borda.append(index)
          continue
        ja_foi_mapeado.add(index)
        pertence_na_grossa = coord_grossa[index]
        index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))

      # Borda x = H z = 0
      for index in range(V * A * (H - 1), (V * A * (H - 1) + (A * (V - 1)) + 1), A):
        if index in ja_foi_mapeado:
          vertices_borda.append(index)
          continue
        ja_foi_mapeado.add(index)
        pertence_na_grossa = coord_grossa[index]
        index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))

      # Borda x = H y = V
      for index in range((V * A * (H - 1)) + (A * (V - 1)), V * A * H):
        if index in ja_foi_mapeado:
          vertices_borda.append(index)
          continue
        ja_foi_mapeado.add(index)
        pertence_na_grossa = coord_grossa[index]
        index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))

      # Borda x = H z = A
      for index in range((V * A * (H - 1)) + (A - 1), V * A * H, A):
        if index in ja_foi_mapeado:
          vertices_borda.append(index)
          continue
        ja_foi_mapeado.add(index)
        pertence_na_grossa = coord_grossa[index]
        index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))

      # Borda y = 0 z = 0
      for index in range(0, V * A * (H - 1) + 1, V * A):
        if index in ja_foi_mapeado:
          vertices_borda.append(index)
          continue
        ja_foi_mapeado.add(index)
        pertence_na_grossa = coord_grossa[index]
        index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))

      # Borda y = 0 z = A
      for index in range(A-1, (V * A * (H - 1)) + A, V * A):
        if index in ja_foi_mapeado:
          vertices_borda.append(index)
          continue
        ja_foi_mapeado.add(index)
        pertence_na_grossa = coord_grossa[index]
        index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))

      # Borda y = V z = 0
      for index in range((V - 1) * A, V * A * (H - 1) + ((V - 1) * A) + 1, V * A):
        if index in ja_foi_mapeado:
          vertices_borda.append(index)
          continue
        ja_foi_mapeado.add(index)
        pertence_na_grossa = coord_grossa[index]
        index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))

      # Borda y = V z = A
      for index in range((V * A) - 1, V * A * H, V * A):
        if index in ja_foi_mapeado:
          vertices_borda.append(index)
          continue
        ja_foi_mapeado.add(index)
        pertence_na_grossa = coord_grossa[index]
        index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))

      # Para cada vértice não borda, andamos para os 6 lados até encontrar um vértice, ou chegar nos limites do volume
      todos_vertices_nao_borda = []
      for index in vertices:
        if index not in vertices_borda:
          todos_vertices_nao_borda.append(index)
      todos_vertices_nao_borda = np.array(todos_vertices_nao_borda)

      for index in todos_vertices_nao_borda:
        y = index // A % V
        z = index % A
        x = index // (A * V)

        # Ando para cima até encontrar um vertice.
        if y < V - 1:
          daVezU = index + A
          while (daVezU not in ja_foi_mapeado):
            pertence_na_grossa = coord_grossa[daVezU]
            index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
            arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([daVezU])))
            ja_foi_mapeado.add(daVezU)
            daVezU += A

        # Ando para baixo
        if y > 0:
          daVezD = index - A
          while (daVezD not in ja_foi_mapeado):
            pertence_na_grossa = coord_grossa[daVezD]
            index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
            arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([daVezD])))
            ja_foi_mapeado.add(daVezD)
            daVezD -= A

        # Ando para a direita
        if x < H - 1:
          daVezR = index + V*A
          while (daVezR not in ja_foi_mapeado):
            pertence_na_grossa = coord_grossa[daVezR]
            index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
            arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([daVezR])))
            ja_foi_mapeado.add(daVezR)
            daVezR += V*A

        # Ando para a esquerda
        if x > 0:
          daVezL = index - V*A
          while (daVezL not in ja_foi_mapeado):
            pertence_na_grossa = coord_grossa[daVezL]
            index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
            arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([daVezL])))
            ja_foi_mapeado.add(daVezL)
            daVezL -= V*A

        # Ando para dentro
        if z < A - 1:
          daVezI = index + 1
          while (daVezI not in ja_foi_mapeado):
            pertence_na_grossa = coord_grossa[daVezI]
            index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
            arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([daVezI])))
            ja_foi_mapeado.add(daVezI)
            daVezI += 1

        # Ando para fora
        if z > 0:
          daVezO = index - 1
          while (daVezO not in ja_foi_mapeado):
            pertence_na_grossa = coord_grossa[daVezO]
            index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
            arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([daVezO])))
            ja_foi_mapeado.add(daVezO)
            daVezO -= 1
      
      # Definir as faces e os internos
      for index in range(V*A*H):
        if index in ja_foi_mapeado:
          continue
        y = index // A % V
        z = index % A
        x = index // (A * V)
        pertence_na_grossa = coord_grossa[index]
        index_na_grossa = pertence_na_grossa[0] * ((A // nZ)) * (V // nY) + pertence_na_grossa[1] * (A // nZ) + pertence_na_grossa[2]
        ja_foi_mapeado.add(index)
        if x == 0 or x == H - 1 or y == 0 or y == V - 1 or z == 0 or z == A - 1:
          faces[index_na_grossa] = np.concatenate((faces[index_na_grossa], np.array([index])))
        else:
          internos[index_na_grossa] = np.concatenate((internos[index_na_grossa], np.array([index])))
      
      self.coord_grossa = coord_grossa
      self.grossa_centroids_array = grossa_centroids_array
      self.vertices = vertices
      self.arestas = arestas
      self.faces = faces
      self.internos = internos
      self.nX = nX
      self.nY = nY
      self.nZ = nZ

    else:
      grossa_centroids_array = np.empty((H // nX) * (V // nY), dtype=object)

      grossa_centroids_array[:] = [np.array([], dtype=int) for _ in range((H // nX) * (V // nY))]

      vertices = np.zeros((H // nX) * (V // nY), dtype=int)

      arestas = np.empty((H // nX) * (V // nY), dtype=object)
      faces = np.empty((H // nX) * (V // nY), dtype=object)

      arestas[:] = [np.array([], dtype=int) for _ in range((H // nX) * (V // nY))]
      faces[:] = [np.array([], dtype=int) for _ in range((H // nX) * (V // nY))]

      # Cria uma grade de valores de i e j
      i_vals = np.arange(H)
      j_vals = np.arange(V)
      i_grid, j_grid = np.meshgrid(i_vals, j_vals, indexing='ij')

      # Calcula indX e indY vetorizados
      indX = i_grid // nX
      indY = j_grid // nY

      # Alterando indX
      if rest_x != 0:
        for i in range(add_x_left):
          indX[nX + i] = 0
        for i in range(nX + add_x_left, H - add_x_right, nX):
          for j in range(0, add_x_left):
            indX[i + nX - (j + 1)] = indX[i]
        for i in range(add_x_right):
          indX[-(i+1)] = indX[-nX]

      # Alterando indY
      if rest_y != 0:
        for x in range(H):
          for i in range(add_y_left):
            indY[x][nY + i] = 0
          for i in range(nY + add_y_left, V - add_y_right, nY):
            for j in range(add_y_left):
              indY[x][i + nY - (j + 1)] = indY[x][i]
          for i in range(add_y_right):
            indY[x][-(i+1)] = indY[x][-nY]

      # Calcula os nós
      nodes = i_grid * V + j_grid

      # Atribui a coordenada grossa para cada nó
      coord_grossa[nodes.flatten()] = list(zip(indX.flatten(), indY.flatten()))

      # Prepara grossa_centroids_array usando uma lista de compreensão
      grossa_centroids_array = np.empty((H // nX) * (V // nY), dtype=object)
      grossa_centroids_array[:] = [np.array([], dtype=int) for _ in range((H // nX) * (V // nY))]

      self.Grossa_Index = np.arange(H*V)
      # Atualiza grossa_centroids_array com os nós, de forma vetorizada
      for indX_val, indY_val, node in zip(indX.flatten(), indY.flatten(), nodes.flatten()):
          idx = indX_val * (V // nY) + indY_val
          self.Grossa_Index[node] = idx
          grossa_centroids_array[idx] = np.append(grossa_centroids_array[idx], node)

      # Calcula x e y para cada índice
      indices = np.arange((H // nX) * (V // nY))
      x = indices // (V // nY)
      y = indices % (V // nY)

      # Calcula os vértices para cada condição
      vertices[(x == 0) & (y == 0)] = [grossa_centroids_array[i][0] for i in indices[(x == 0) & (y == 0)]]
      vertices[(x == (H // nX - 1)) & (y == 0)] = [grossa_centroids_array[i][-(nY + add_y_left)] for i in indices[(x == (H // nX - 1)) & (y == 0)]]
      vertices[(x == 0) & (y == (V // nY - 1))] = [grossa_centroids_array[i][(nY + add_y_right - 1)] for i in indices[(x == 0) & (y == (V // nY - 1))]]
      vertices[(x == (H // nX - 1)) & (y == (V // nY - 1))] = [grossa_centroids_array[i][-1] for i in indices[(x == (H // nX - 1)) & (y == (V // nY - 1))]]
      vertices[(x == 0) & (y > 0) & (y < (V // nY - 1))] = [grossa_centroids_array[i][nY // 2] for i in indices[(x == 0) & (y > 0) & (y < (V // nY - 1))]]
      vertices[(y == (V // nY - 1)) & (x > 0) & (x < (H // nX - 1))] = [grossa_centroids_array[i][(nY + add_y_right) * (nX // 2 + 1) - 1] for i in indices[(y == (V // nY - 1)) & (x > 0) & (x < (H // nX - 1))]]
      vertices[(y == 0) & (x > 0) & (x < (H // nX - 1))] = [grossa_centroids_array[i][(nY + add_y_left) * (nX // 2)] for i in indices[(y == 0) & (x > 0) & (x < (H // nX - 1))]]
      vertices[(x == (H // nX - 1)) & (y > 0) & (y < (V // nY - 1))] = [grossa_centroids_array[i][-(nY // 2 + 1)] for i in indices[(x == (H // nX - 1)) & (y > 0) & (y < (V // nY - 1))]]
      vertices[(x > 0) & (x < (H // nX - 1)) & (y > 0) & (y < (V // nY - 1))] = [grossa_centroids_array[i][(nX * nY) // 2] for i in indices[(x > 0) & (x < (H // nX - 1)) & (y > 0) & (y < (V // nY - 1))]]

      # Adiciona valores ao conjunto
      ja_foi_mapeado = set(vertices)

      # Preciso pegar as arestas
      # Borda lateral esquerda
      for index in range(0, V):
        if index in ja_foi_mapeado:
          continue
        ja_foi_mapeado.add(index)
        pertence_na_grossa = coord_grossa[index]
        index_na_grossa = pertence_na_grossa[0]*((V // nY)) + pertence_na_grossa[1]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))

      # Borda inferior
      for index in range(0, H*V, V):
        if index in ja_foi_mapeado:
          continue
        ja_foi_mapeado.add(index)
        pertence_na_grossa = coord_grossa[index]
        index_na_grossa = pertence_na_grossa[0]*((V // nY)) + pertence_na_grossa[1]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))


      # Borda superior
      for index in range(V-1, H*V, V):
        if index in ja_foi_mapeado:
          continue
        ja_foi_mapeado.add(index)
        pertence_na_grossa = coord_grossa[index]
        index_na_grossa = pertence_na_grossa[0]*((V // nY)) + pertence_na_grossa[1]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))


      # Borda lateral direita
      for index in range(V*(H-1), H*V):
        if index in ja_foi_mapeado:
          continue
        ja_foi_mapeado.add(index)
        pertence_na_grossa = coord_grossa[index]
        index_na_grossa = pertence_na_grossa[0]*((V // nY)) + pertence_na_grossa[1]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))


      #Para cada vertice não-borda, só andar pros 4 lados até atingir um vértice.

      todos_vertices = vertices
      todos_vertices_nao_borda = []

      for index in range(0, (H // nX) * (V // nY)):
        i = index // ((V // nY))
        j = index % ((V // nY))
        if(i == 0 or i == (H//nX - 1) or j == 0 or j == (V // nY - 1)):
          continue
        todos_vertices_nao_borda.append(vertices[index])

      todos_vertices_nao_borda = np.array(todos_vertices_nao_borda)

      for index in todos_vertices_nao_borda:
        # Ando para cima até encontrar um vertice.
        daVezU = index + 1

        while (daVezU not in ja_foi_mapeado):
          pertence_na_grossa = coord_grossa[daVezU]
          index_na_grossa = pertence_na_grossa[0]*((V // nY)) + pertence_na_grossa[1]
          arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([daVezU])))
          ja_foi_mapeado.add(daVezU)
          daVezU += 1

        # Ando para baixo
        daVezD = index - 1
        while (daVezD not in ja_foi_mapeado):
          pertence_na_grossa = coord_grossa[daVezD]
          index_na_grossa = pertence_na_grossa[0]*((V // nY)) + pertence_na_grossa[1]
          arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([daVezD])))
          ja_foi_mapeado.add(daVezD)
          daVezD -= 1

        # Ando para a direita
        daVezR = index + V
        while (daVezR not in ja_foi_mapeado):
          pertence_na_grossa = coord_grossa[daVezR]
          index_na_grossa = pertence_na_grossa[0]*((V // nY)) + pertence_na_grossa[1]
          arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([daVezR])))
          ja_foi_mapeado.add(daVezR)
          daVezR += V

        # Ando para a esquerda
        daVezL = index - V
        while (daVezL not in ja_foi_mapeado):
          pertence_na_grossa = coord_grossa[daVezL]
          index_na_grossa = pertence_na_grossa[0]*((V // nY)) + pertence_na_grossa[1]
          arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([daVezL])))
          ja_foi_mapeado.add(daVezL)
          daVezL -= V

      for index in range(0, V*H):
        if(index not in ja_foi_mapeado):
          ja_foi_mapeado.add(index)
          pertence_na_grossa = coord_grossa[index]
          index_na_grossa = pertence_na_grossa[0]*((V // nY)) + pertence_na_grossa[1]
          faces[index_na_grossa] = np.concatenate((faces[index_na_grossa], np.array([index])))

      self.coord_grossa = coord_grossa
      self.grossa_centroids_array = grossa_centroids_array
      self.vertices = vertices
      self.arestas = arestas
      self.faces = faces
      self.nX = nX
      self.nY = nY

  def process(self, nX, nY, nZ = None, nodes = None, connections = None):
    self.set_nodes(nodes)
    self.set_connectivities(connections)
    self.get_centroids()
    self.centroids = self.sort_centroids(self.centroids)
    self.get_dimensions()
    self.divide_mesh(self.centroids, self.nHorizontal, self.nVertical, nX, nY, self.nAltura, nZ)
    print("Process  Ended")

Classe para armazenar mais de um nível e separar os IDs da NU-ADM (Ainda nada eficiente)

Fazer a atribuição dos ids da mais grossa para a mais fina para ver como fica

In [19]:
class NUADM_ID:
    def __init__(self, nX_all, nY_all, nZ_all, niveis = 2, dim = 3):
        self.levels = niveis
        self.mesh_level = []
        self.dimension = dim
        self.nX = nX_all
        self.nY = nY_all
        self.nZ = nZ_all
        self.index_atual = 0
        self.ids = []
        self.id_NUADM = []
        self.mapping = []
    
    def generate_levels(self, nodes, connections):
        if self.dimension == 3:
            first = MeshDataStructure('', '', self.dimension)
            first.process(self.nX[0], self.nY[0], self.nZ[0], nodes, connections)
            self.mesh_level.append(first)
            nextH = self.mesh_level[0].nHorizontal
            nextV = self.mesh_level[0].nVertical
            nextA = self.mesh_level[0].nAltura
            # Inicializa o mapeamento de indices de nivel 0: fina -> grossa 0 (já existe o mapeamento para o nivel 0)
            self.mapping.append(self.mesh_level[0].Grossa_Index)
            
            for i in range(self.levels - 2):
                nextH = nextH // self.nX[i]
                nextV = nextV // self.nY[i]
                nextA = nextA // self.nZ[i]
                centroids_grossa = np.arange(nextV * nextA * nextH)
                new = MeshDataStructure('', '', self.dimension)
                new.divide_mesh(centroids_grossa, nextH, nextV, self.nX[i+1], self.nY[i+1], nextA, self.nZ[i+1])
                self.mesh_level.append(new)

                # criando o mapeamento de indices de níveis > 0: fina -> grossa 0 -> grossa 1...
                aux_vet = np.array([-1] * len(self.mesh_level[0].centroids))
                for j in range(len(self.mesh_level[0].centroids)):
                    index = self.mapping[0][j]
                    for k in range(i+1):
                        index = self.mesh_level[k+1].Grossa_Index[index]
                    aux_vet[j] = index
                self.mapping.append(aux_vet)
                
        if self.dimension == 2:
            first = MeshDataStructure('', '', self.dimension)
            first.process(self.nX[0], self.nY[0], nodes = nodes, connections=connections)
            self.mesh_level.append(first)
            nextH = self.mesh_level[0].nHorizontal
            nextV = self.mesh_level[0].nVertical
            for i in range(self.levels - 2):
                nextH = nextH // self.nX[i]
                nextV = nextV // self.nY[i]
                centroids_grossa = np.arange(nextV * nextH)
                new = MeshDataStructure('', '', self.dimension)
                new.divide_mesh(centroids_grossa, nextH, nextV, self.nX[i+1], self.nY[i+1])
                self.mesh_level.append(new)
        print('done generating levels')

    
    def generate_ids(self, level_vector):
        # inicializando vetor de ids NU-ADM
        level_vector = np.array(level_vector)
        self.id_NUADM = np.full(len(level_vector), -1)
        self.ids = [np.full(len(lvl.grossa_centroids_array), -1) for lvl in self.mesh_level]
        print('ids initialized')
        
        index_0 = np.where(level_vector == 0)[0]
        self.id_NUADM[index_0] = np.arange(len(index_0))
        self.index_atual += len(index_0)

        for level in range(1, self.levels):
            index_level = np.where(level_vector == level)[0]
            if len(index_level) == 0:
                continue

            for index in index_level:

                indice = self.mapping[level - 1][index]

                if self.ids[level - 1][indice] == -1:
                    self.ids[level - 1][indice] = self.index_atual
                    self.index_atual += 1
                
                self.id_NUADM[index] = self.ids[level - 1][indice]

    def validate(self):
        print(f'levels: {self.levels}')
        print(f'level 0: {len(self.mesh_level[0].centroids)}')
        for i in range(self.levels - 1):
            print(f'level {i+1}: {len(self.mesh_level[i].grossa_centroids_array)}')


In [17]:
import random
vector = []
for i in range(30**2):
    vector.append(random.randint(0, 2))

In [20]:
nodes, elements = Read_Mesh('Malhas_teste_3D/semi_45_45_45.msh', 3)
# print (vector)
nX = [3, 3, 3]
nY = [3, 3, 3]
nZ = [3, 3, 3]
sla = NUADM_ID(nX, nY, nZ, niveis=3, dim=3)
sla.generate_levels(nodes, elements)
init = time.time()
sla.generate_ids(vector)
fim = time.time()
print(fim - init)
sla.validate()
print(sla.id_NUADM)
np.save('nuadm_id.npy', sla.id_NUADM)
np.save('level.npy', vector)


Tempo de leitura da malha: 0.8650 segundos
Process  Ended
done generating levels
ids initialized
0.0460965633392334
levels: 3
level 0: 91125
level 1: 3375
level 2: 125
[403   0   1 303   2 303 403 304 403 305 305 404   3 306 306 307 404   4
 308 405 308 309 405 309 405 310 310 406 406 406   5 311 311   6 406 312
 313   7 313   8 314 407   9  10 315 403 403 316 403 303 403  11  12 403
 404 305  13  14  15  16 307 404 307  17 405  18 309 309 405 405 310 310
 406 317 406 311 406  19 312 312 406 407 407  20 407 314 314 315  21  22
  23 316 403  24 403 303 304  25 304 404  26  27 404 404  28 307 404  29
  30 308 405 309  31 405  32  33 310  34 317 317  35 311 311 406 312 406
  36 313  37 407 407  38 315 407 315  39 403  40 403 318 318  41 319 403
  42  43 320  44  45 404 404 404 321  46  47  48  49 405 405 405 405  50
 406  51 322  52 406 406 406 323 406 324 407  53  54 407 407 325 325 325
  55 403 403  56 318 318 319 319 403  57 404  58 404 404  59  60 404 321
 405 326 326 405 327 405 405

Código para exportar a malha dual para ver no visit (lembrar de alterar no arquivo do docker também)

In [7]:
# nodes, elements = Read_Mesh('Malhas_teste_3D/semi_106_95_104.msh', 3)
# print('Processando a malha fina')
# teste = MeshDataStructure('','', 3)
# teste.process(3, 3, 3, nodes, elements)
# print(teste.nHorizontal, teste.nVertical, teste.nAltura)

# print('Salvando malha dual 1')
# dual = np.zeros(len(teste.connectivities), dtype = int)
# for vertice in teste.vertices:
#     dual[vertice] = 3
# for aresta in teste.arestas:
#     for a in aresta:
#         dual[a] = 2
# for face in teste.faces:
#     for f in face:
#         dual[f] = 1
# np.save('Malha_Dual.npy', dual)
# print('Ok')

# # Montando segundo nível
# print("Inicio segundo nivel")
# newH = teste.nHorizontal // teste.nX
# newV = teste.nVertical // teste.nY
# newA = teste.nAltura // teste.nZ
# centroids_grossa = np.arange(newV * newA * newH)
# new = MeshDataStructure('', '', 3)
# new.divide_mesh(centroids_grossa, newH, newV, 3, 3, newA, 3)
# print('Segundo nivel ok')
# print(newH, newV, newA)

# print('Salvando malha dual 2')
# dual = np.zeros(len(teste.connectivities), dtype = int)
# for vertice in new.vertices:
#     dual[teste.grossa_centroids_array[vertice]] = 3
# for aresta in new.arestas:
#     for a in aresta:
#         dual[teste.grossa_centroids_array[a]] = 2
# for face in new.faces:
#     for f in face:
#         dual[teste.grossa_centroids_array[f]] = 1
# np.save('Malha_Dual_2.npy', dual)
# print('Ok')

# # Montando terceiro nível
# print("Inicio terceiro nivel")
# newH = newH // new.nX
# newV = newV // new.nY
# newA = newA // new.nZ
# centroids_grossa = np.arange(newV * newA * newH)
# novo = MeshDataStructure('', '', 3)
# novo.divide_mesh(centroids_grossa, newH, newV, 3, 3, newA, 3)
# print('Terceiro nivel ok')
# print(newH, newV, newA)
# print(len(novo.grossa_centroids_array))

# print('Salvando malha dual 3')
# dual = np.zeros(len(teste.connectivities), dtype = int)
# for vertice in novo.vertices:
#     for v in new.grossa_centroids_array[vertice]:
#         dual[teste.grossa_centroids_array[v]] = 3
# for aresta in novo.arestas:
#     for a in aresta:
#         for ar in new.grossa_centroids_array[a]:
#             dual[teste.grossa_centroids_array[ar]] = 2
# for face in novo.faces:
#     for f in face:
#         for fac in new.grossa_centroids_array[f]:
#             dual[teste.grossa_centroids_array[fac]] = 1
# np.save('Malha_Dual_3.npy', dual)
# print('Ok')

# Montando quarto nível
# print("Inicio quarto nivel")
# newH = newH // novo.nX
# newV = newV // novo.nY
# newA = newA // novo.nZ
# centroids_grossa = np.arange(newV * newA * newH)
# prox = MeshDataStructure('', '', 3)
# prox.divide_mesh(centroids_grossa, newH, newV, 3, 3, newA, 3)
# print('Quarto nivel ok')
# print(newH, newV, newA)

# print(len(prox.grossa_centroids_array))
# print('Salvando malha dual 3')
# dual = np.zeros(len(teste.connectivities), dtype = int)
# for vertice in prox.vertices:
#     for v in prox.grossa_centroids_array[vertice]:
#         dual[teste.grossa_centroids_array[v]] = 3
# for aresta in prox.arestas:
#     for a in aresta:
#         for ar in prox.grossa_centroids_array[a]:
#             dual[teste.grossa_centroids_array[ar]] = 2
# for face in prox.faces:
#     for f in face:
#         for fac in prox.grossa_centroids_array[f]:
#             dual[teste.grossa_centroids_array[fac]] = 1
# np.save('Malha_Dual_4.npy', dual)
# print('Ok')